In [1]:
%pip install datasets transformers 'accelerate>=0.26.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.6/562.6 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 98.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset   
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import numpy as np
import torch

dsn = "cubbk/rixvox-tokenised"
model_name = "canopylabs/orpheus-tts-0.1-pretrained"

# Training Args
epochs = 1
batch_size = 1
number_processes = 1
pad_token = 128263
save_steps = 5000
learning_rate = 5.0e-5

# Naming and paths
save_folder = "checkpoints"
project_name = "tuning-orpheus"
run_name = "5e5-0"

# Ensure bf16 only when supported
bf16_supported = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
dtype = torch.bfloat16 if bf16_supported else torch.float32
if not bf16_supported:
    print("bfloat16 not supported on this device; using float32.")

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = pad_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=dtype,
)

raw_ds = load_dataset(dsn, split="train")
split = raw_ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = split["train"], split["test"]


training_args = TrainingArguments(
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    logging_steps=10,
    bf16=bf16_supported,
    output_dir=f"./{save_folder}",
    # report_to="wandb",
    save_steps=save_steps,
    remove_unused_columns=True,
    learning_rate=learning_rate,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds, # type: ignore
    eval_dataset=eval_ds, # type: ignore
)

trainer.train()


tokenizer_config.json:   0%|          | 0.00/5.41M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

README.md:   0%|          | 0.00/354 [00:00<?, ?B/s]

dev/train-00000-of-00001.parquet:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7933 [00:00<?, ? examples/s]

Step,Training Loss
10,4.926600
20,4.766500
30,4.713700
40,4.648200
50,4.620900
60,4.543200
70,4.791600
80,4.591600
90,4.517300
100,4.560100


TrainOutput(global_step=7536, training_loss=4.164014416388898, metrics={'train_runtime': 4850.8499, 'train_samples_per_second': 1.554, 'train_steps_per_second': 1.554, 'total_flos': 2.4539326456638874e+17, 'train_loss': 4.164014416388898, 'epoch': 1.0})

In [5]:
# import math
# eval_results = trainer.evaluate()

# perplexity = math.exp(eval_results["eval_loss"])
# print("Eval loss:", eval_results["eval_loss"])
# print("Perplexity:", perplexity)

In [6]:
# Save final model & tokenizer
save_path = "./final-model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./final-model/tokenizer_config.json',
 './final-model/special_tokens_map.json',
 './final-model/chat_template.jinja',
 './final-model/tokenizer.json')

In [7]:
reloaded = AutoModelForCausalLM.from_pretrained(save_path, torch_dtype=dtype)
_ = reloaded.generate(**tokenizer("Test:", return_tensors="pt"), max_new_tokens=5)
print("Reload OK.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Reload OK.


In [9]:
prompt = "Hej, hur mår du idag?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
model.eval()
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.8,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
print(out)

tensor([[128000,   1548,     73,     11,  13113,    296,  18382,   3930,    887,
            351,     30,  93212,  19106,   3930,  17053,     30,  30511,   4960,
            294,   2357,   8212,   9603,   1651,  18619,   2629,   8261,    959,
          17053,  27822,    296,  15492,   1794,  13728,  63901,    357,   6312,
          10382,  12218,   1794,   9077,   6520,    665,    348,  97533,    357,
           6312,  10382,  64786,    602,  29169,     13,    328,   3870,  19106,
          27840,  50772,    528,    676,    261,   1037,   1651]],
       device='cuda:0')


In [10]:
%pip install huggingface_hub
# huggingface-cli login
reloaded.push_to_hub("cubbk/orpheus-swedish")
tokenizer.push_to_hub("cubbk/orpheus-swedish")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpalqfmtnt/tokenizer.json       :  99%|#########8| 22.5MB / 22.8MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/cubbk/orpheus-swedish/commit/252757be8206012e6305a4aa51b4cf514c64ad13', commit_message='Upload tokenizer', commit_description='', oid='252757be8206012e6305a4aa51b4cf514c64ad13', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cubbk/orpheus-swedish', endpoint='https://huggingface.co', repo_type='model', repo_id='cubbk/orpheus-swedish'), pr_revision=None, pr_num=None)